In [27]:
#All Required Imports 
import os
import datetime
now = datetime.datetime.now
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbn # better plotting and aesthetics
from pathlib import Path # just a utility for better cross-platform file-loading
from scipy.io import loadmat
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

import sklearn.neural_network as nn

import tensorflow.keras
from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.optimizers import Adam

import pandas as pd
import seaborn as sbn  # this is a nice plotting library
import warnings

from tensorflow.keras import losses, optimizers
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization  # we have 2D images
from tensorflow.keras.layers import Dense, MaxPool2D, LeakyReLU, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

# from tensorflow.keras.utils import to_categorical

from keras.utils import np_utils

from tensorflow.keras.layers import Dropout

# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

batch_size = 128

# Number of classes where training is done on all layers
num_train_classes = 7

# Number of classes where training is not done on all layers
num_transfer_learn_classes = 3

epochs = 10

#Reference
#https://keras.io/examples/mnist_transfer_cnn/

In [24]:
def train_model(model, x_train, y_train, x_test, y_test):
   

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    train_start_time = now()
    model.fit(
    x_train,
    y_train,
    epochs=epochs,
    verbose=1,  # show progress bar
)
    print('Training time: %s' % (now() - train_start_time))
    score = model.evaluate(x_test, y_test, verbose=0)   #verbose=0 to avoid unwanted outputs
    print('Test Loss:', score[0])
    print('Test accuracy:', score[1])

    
    

In [25]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [26]:
# Bring the input data between 0 and 1
x_train = x_train/255
x_test = x_test/255

no_of_samples, data_dim1, data_dim2 = x_train.shape
#Reshape to have each sample data in grey scale Image format
x_train = np.reshape(x_train, (no_of_samples, data_dim1, data_dim2, 1))
x_train = x_train.astype('float32')

no_of_samples, data_dim1, data_dim2 = x_test.shape
#Reshape to have each sample data in grey scale Image format
x_test = np.reshape(x_test, (no_of_samples, data_dim1, data_dim2, 1))

x_test = x_test.astype('float32')


# create two datasets one with digits below 7 and one with 7 and above
x_train_lt7 = x_train[y_train < 7]
y_train_lt7 = y_train[y_train < 7]
x_test_lt7 = x_test[y_test < 7]
y_test_lt7 = y_test[y_test < 7]


x_train_gte7 = x_train[y_train >= 7]
y_train_gte7 = y_train[y_train >= 7] 
x_test_gte7 = x_test[y_test >= 7]
y_test_gte7 = y_test[y_test >= 7] 


# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(32, kernel_size,
           padding='valid',
           input_shape=(28,28,1)),
    Activation('relu'),
    Conv2D(32, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Conv2D(128, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
]

classification_layers = [
    Dense(64),
    Activation('relu'),
    Dropout(0.25),
    Dense(16),
    Activation('relu'),
    Dense(num_train_classes),
    Activation('softmax')
]


# convert class vectors to one hot encoding using Panda
s = pd.Series(y_train_lt7)
y_train_lt7 = pd.get_dummies(s)
y_train_lt7 = y_train_lt7.values

s = pd.Series(y_test_lt7)
y_test_lt7 = pd.get_dummies(s)
y_test_lt7 = y_test_lt7.values


s = pd.Series(y_train_gte7)
y_train_gte7 = pd.get_dummies(s)
y_train_gte7 = y_train_gte7.values

s = pd.Series(y_test_gte7)
y_test_gte7 = pd.get_dummies(s)
y_test_gte7 = y_test_gte7.values

# create complete model
model = Sequential(feature_layers + classification_layers)


# train the whole model for 7-digit classification [0..7]
train_model(model, x_train_lt7, y_train_lt7, x_test_lt7, y_test_lt7)

# freeze feature layers and rebuild model to have the learning transfered to the new model
trans_learning_model = Sequential()

for i in range(len(feature_layers)):
    layer = model.layers[i]
    layer.trainable = False
    trans_learning_model.add(layer)
    

# Add Trainable Layers    
trans_learning_model.add((Dense(units=128, activation="relu")))
trans_learning_model.add(Dropout(0.1))
trans_learning_model.add(Dense(units=16, activation="relu"))
trans_learning_model.add(Dense(num_transfer_learn_classes, activation='softmax'))

# train model for 7, 8 and 9
train_model(trans_learning_model, x_train_gte7, y_train_gte7, x_test_gte7, y_test_gte7)



Train on 41935 samples
Epoch 1/10
41935/41935 [==============================] - 81s 2ms/sample - loss: 0.1591 - accuracy: 0.9494
Epoch 2/10
41935/41935 [==============================] - 80s 2ms/sample - loss: 0.0505 - accuracy: 0.9862
Epoch 3/10
41935/41935 [==============================] - 84s 2ms/sample - loss: 0.0388 - accuracy: 0.9886
Epoch 4/10
41935/41935 [==============================] - 89s 2ms/sample - loss: 0.0299 - accuracy: 0.9917
Epoch 5/10
41935/41935 [==============================] - 83s 2ms/sample - loss: 0.0255 - accuracy: 0.9921
Epoch 6/10
41935/41935 [==============================] - 83s 2ms/sample - loss: 0.0215 - accuracy: 0.9932
Epoch 7/10
41935/41935 [==============================] - 90s 2ms/sample - loss: 0.0200 - accuracy: 0.9938
Epoch 8/10
41935/41935 [==============================] - 93s 2ms/sample - loss: 0.0190 - accuracy: 0.9943
Epoch 9/10
41935/41935 [==============================] - 87s 2ms/sample - loss: 0.0155 - accuracy: 0.9953
Epoch 10/10
41